In [1]:
!pip install pulp

Defaulting to user installation because normal site-packages is not writeable


In [4]:
import random
from itertools import permutations
from pulp import LpProblem, LpVariable, LpStatus, LpMinimize, lpSum
from typing import Dict, List, Tuple
from pulp import GLPK_CMD

def generate_dummy_data(n_pois: int, n_days: int) -> tuple:
    """
    Generate dummy data for the itinerary optimization problem.

    Parameters:
    - n_pois: The number of points of interest (POIs).
    - n_days: The number of days.

    Returns:
    - pois: A list of POI names.
    - days: A list of day identifiers.
    - travel_time: A dictionary with travel times between each pair of POIs.
    """
    pois = [f'poi_{i}' for i in range(n_pois)]
    days = [f'D{day}' for day in range(1, n_days + 1)]
    travel_time = {(poi1, poi2): random.randint(10, 60) for poi1, poi2 in permutations(pois, 2)}
    return pois, days, travel_time



def extract_itinerary(nodes):
    # Step 1: Group nodes by day
    day_edges = {}
    for node in nodes:
        poi1, poi2, day = node
        if day not in day_edges:
            day_edges[day] = []
        day_edges[day].append((poi1, poi2))
    
    itineraries = {}
    # Step 2: Construct the graph for each day and find the path
    for day, edges in day_edges.items():
        graph = {}
        in_degree = {}  # Track incoming edges
        for poi1, poi2 in edges:
            graph[poi1] = poi2
            # Update in_degree counts
            in_degree[poi2] = in_degree.get(poi2, 0) + 1
            if poi1 not in in_degree:
                in_degree[poi1] = 0
        
        # Step 3: Find the starting POI (no incoming edges) and construct the itinerary
        start_poi = None
        for poi, in_deg in in_degree.items():
            if in_deg == 0:
                start_poi = poi
                break
        
        itinerary = [start_poi]
        while start_poi in graph:
            next_poi = graph[start_poi]
            itinerary.append(next_poi)
            start_poi = next_poi
        
        itineraries[day] = itinerary
    
    return itineraries

In [13]:
# Initialize data
n_pois = 100  # Number of POIs
n_days = 20  # Number of days
pois, days, travel_time = generate_dummy_data(n_pois, n_days)

# Initialize the problem
problem = LpProblem("Itinerary_Optimization", LpMinimize)

# Variables
order_vars = {
    (poi1, poi2, day): LpVariable(f"order_{poi1}_{poi2}_{day}", cat="Binary")
    for poi1 in pois + ['nonce_node'] for poi2 in pois + ['nonce_node'] 
    for day in days if poi1 != poi2
}
# Objective function: Minimize the total travel time
problem += lpSum(
    travel_time[(poi_from, poi_to)] * order_vars[(poi_from, poi_to, day)]
    for poi_from, poi_to in permutations(pois, 2) for day in days
)

# Constraints

# Each POI visited exactly once across all days
for poi in pois:
    problem += lpSum(
        order_vars[(poi, other_poi, day)] + order_vars[(other_poi, poi, day)]
        for day in days for other_poi in pois + ['nonce_node'] if poi != other_poi
    ) == 2, f"enter_and_exit_{poi}"

# Ensure that starting nonce_nodes are visited n_days times
problem += lpSum(order_vars[('nonce_node', poi, day)] 
                 for poi in pois for day in days) == n_days, "start_nonce"

# Ensure that ending nonce_nodes are visited n_days times
problem += lpSum(order_vars[(poi, 'nonce_node', day)] 
                 for poi in pois for day in days) == n_days, "end_nonce"

for day in days:
    problem += lpSum(
        order_vars[('nonce_node', poi, day)] 
        for poi in pois) == 1, f"start_nonce_{day}"
    problem += lpSum(
        order_vars[(poi, 'nonce_node', day)] 
        for poi in pois) == 1, f"end_nonce_{day}"


# Ensure entering and exiting each POI occurs within the same day
for day in days:
    for poi in pois:
        problem += lpSum(
            order_vars[(poi, other_poi, day)]
            for other_poi in pois + ['nonce_node'] if poi != other_poi
        ) == lpSum(
            order_vars[(other_poi, poi, day)]
            for other_poi in pois + ['nonce_node'] if poi != other_poi
        ), f"same_day_visit_{poi}_{day}"


# problem
# Solve the problem
status = problem.solve(GLPK_CMD(msg=0))

# Check the status and print the results
if LpStatus[status] == 'Optimal':
    print("Solution Found:\n")
    results = []
    for var in order_vars:
        if order_vars[var].varValue == 1:
            results.append(var)
            print(f"{var} = {order_vars[var].varValue}")
else:
    print("No optimal solution found.")


Solution Found:

('poi_0', 'nonce_node', 'D15') = 1
('poi_1', 'nonce_node', 'D13') = 1
('poi_2', 'poi_55', 'D20') = 1
('poi_3', 'poi_30', 'D15') = 1
('poi_4', 'poi_91', 'D4') = 1
('poi_5', 'poi_53', 'D18') = 1
('poi_6', 'poi_57', 'D10') = 1
('poi_7', 'poi_62', 'D9') = 1
('poi_8', 'poi_41', 'D5') = 1
('poi_9', 'poi_61', 'D5') = 1
('poi_10', 'poi_7', 'D9') = 1
('poi_11', 'poi_16', 'D4') = 1
('poi_12', 'poi_0', 'D15') = 1
('poi_13', 'poi_76', 'D15') = 1
('poi_14', 'poi_90', 'D19') = 1
('poi_15', 'nonce_node', 'D18') = 1
('poi_16', 'poi_4', 'D4') = 1
('poi_17', 'nonce_node', 'D2') = 1
('poi_18', 'poi_94', 'D11') = 1
('poi_19', 'poi_6', 'D10') = 1
('poi_20', 'poi_26', 'D5') = 1
('poi_21', 'poi_65', 'D6') = 1
('poi_22', 'poi_31', 'D20') = 1
('poi_23', 'poi_13', 'D15') = 1
('poi_24', 'poi_42', 'D9') = 1
('poi_25', 'poi_27', 'D15') = 1
('poi_26', 'nonce_node', 'D5') = 1
('poi_27', 'poi_50', 'D15') = 1
('poi_28', 'poi_92', 'D15') = 1
('poi_29', 'nonce_node', 'D16') = 1
('poi_30', 'poi_25', 'D15